# Projet Vision par Ordinateur

In [ ]:

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix
import seaborn as sns


## 1. Chargement du dataset CIFAR-10 et préparation

In [ ]:

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

print("Shape X_train:", x_train.shape)
print("Shape y_train:", y_train.shape)


## 2. Répartition des classes

In [ ]:

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']
counts = np.bincount(y_train.flatten())

plt.figure(figsize=(10,6))
plt.bar(classes, counts)
plt.title("Répartition des classes dans CIFAR-10")
plt.xticks(rotation=45)
plt.show()


## 3. Vérification des images et labels

In [ ]:

idx = np.random.choice(len(x_train), 9, replace=False)
plt.figure(figsize=(10,10))
for i, index in enumerate(idx):
    plt.subplot(3, 3, i+1)
    plt.imshow(x_train[index])
    plt.title(classes[y_train[index][0]])
    plt.axis('off')
plt.show()


## 4. Construction et entraînement du modèle CNN

In [ ]:

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(patience=3, restore_best_weights=True)

history = model.fit(x_train, y_train_cat, epochs=10, batch_size=64,
                    validation_split=0.2, callbacks=[early_stop])


## 5. Analyse des performances - courbes

In [ ]:

plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.title("Loss")
plt.show()

plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.title("Accuracy")
plt.show()


## 6. Prédiction, accuracy et matrice de confusion

In [ ]:

loss, acc = model.evaluate(x_test, y_test_cat)
print("Test accuracy:", acc)

y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = y_test.flatten()

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=classes, yticklabels=classes, cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Matrice de confusion")
plt.show()


## 7. Sauvegarde du modèle

In [ ]:

model.save("cifar10_model.h5")
